In [1]:
import openclsim.apps as apps
import openclsim.model as model
import openclsim.plugins as plugins

import shapely
import scipy.stats as st
import plotly.graph_objects as go

import numpy as np

In [24]:
class BaseModel(apps.SimulationEnvironment):
    
    # Activity lengths
    LOAD_COMPONENT = 3600
    TRANSIT_TO = 3600
    POSITIONING = 1800
    INSTALL_COMPONENT = 5 * 3600
    TRANSIT_FROM = 3600 + 1800

    def define_offshore_environment(self):
        pass

    def define_entities(self):

        self.port = apps.Site(
            env=self.env,
            name="PORT",
            geometry=shapely.geometry.Point(0, 0),
            capacity=10,
            level=10,
            nr_resources=1
        )

        self.owf = apps.Site(
            env=self.env,
            name="OWF",
            geometry=shapely.geometry.Point(1, 1),
            capacity=10,
            level=0,
            nr_resources=1
        )

        self.aeolus = apps.InstallationEquipment(
            env=self.env,
            name="AEOLUS",
            geometry=self.port.geometry,
            capacity=4,
            level=0,
            nr_resources=1,
            unloading_rate=1/3600,
            loading_rate=1/3600,
            compute_v=lambda x: 10
        )

        # Return a list of all the entities
        return [self.port, self.aeolus, self.owf]

    def define_operation(self):

        # Base activities
        load_component = model.ShiftAmountActivity(
            env=self.env,
            registry=self.registry,
            name="TRANSFER COMPONENT TO DECK.",
            processor=self.aeolus,
            origin=self.port,
            destination=self.aeolus,
            duration=BaseModel.LOAD_COMPONENT,
            amount=1
        )

        transit_to_owf = model.MoveActivity(
            env=self.env,
            registry=self.registry,
            name="TRANSIT TO OFFSHORE SITE.",
            mover=self.aeolus,
            destination=self.owf,
            duration=BaseModel.TRANSIT_TO
        )

        position = model.BasicActivity(
            env=self.env,
            registry=self.registry,
            name="(RE)POSITION ONTO LOCATION.",
            duration=BaseModel.POSITIONING
        )

        install_component = model.ShiftAmountActivity(
            env=self.env,
            registry=self.registry,
            name="INSTALL COMPONENT ON SITE.",
            processor=self.aeolus,
            origin=self.aeolus,
            destination=self.owf,
            duration=BaseModel.INSTALL_COMPONENT,
            amount=1
        )

        transit_to_port = model.MoveActivity(
            env=self.env,
            registry=self.registry,
            name="TRANSIT TO PORT.",
            mover=self.aeolus,
            destination=self.port,
            duration=BaseModel.TRANSIT_FROM
        )

        
        # Loading cycle
        loading_cycle = model.WhileActivity(
            env=self.env,
            registry=self.registry,
            name="LOAD CYCLE.",
            sub_processes=[load_component],
            condition_event={"or": [
                {"type": "container", "concept": self.aeolus, "state": "full"},
                {"type": "container", "concept": self.port, "state": "empty"},
            ]}
        )

        # Unload/Installation cycle
        installation_cycle = model.WhileActivity(
            env=self.env,
            registry=self.registry,
            name="INSTALL CYCLE",
            sub_processes=[position, install_component],
            condition_event={"or": [
                {"type": "container", "concept": self.aeolus, "state": "empty"},
                {"type": "container", "concept": self.owf, "state": "full"},
            ]}
        )

        # Outer installation cycle
        outer_cycle = model.WhileActivity(
            env=self.env,
            registry=self.registry,
            name="OUTER CYCLE",
            sub_processes=[
                loading_cycle,
                transit_to_owf,
                installation_cycle,
                transit_to_port
            ],
            condition_event={"or": [
                {"type": "container", "concept": self.port, "state": "empty"},
                {"type": "container", "concept": self.owf, "state": "full"}
            ]}
        )

        # List base activities and store them as a property
        self.activities = [
            load_component, 
            transit_to_owf,
            position,
            install_component,
            transit_to_port
        ]
        
        # Return upstream activity
        return outer_cycle

In [25]:
x = BaseModel(start_date='0')
x.initiate_simulation()

'SUCCESSFUL.'

In [26]:
x.event_log.drop("ActivityID", axis=1)

,Timestamp,ActivityState,Description,PORT,AEOLUS,OWF
0,2020-01-01 00:00:00,START,TRANSFER COMPONENT TO DECK.,"{'container level': 10, 'geometry': POINT (0 0)}","{'geometry': POINT (0 0), 'container level': 0}","{'container level': 0, 'geometry': POINT (1 1)}"
1,2020-01-01 01:00:00,STOP,TRANSFER COMPONENT TO DECK.,"{'container level': 9, 'geometry': POINT (0 0)}","{'geometry': POINT (0 0), 'container level': 1}","{'container level': 0, 'geometry': POINT (1 1)}"
2,2020-01-01 01:00:00,START,TRANSFER COMPONENT TO DECK.,"{'container level': 9, 'geometry': POINT (0 0)}","{'geometry': POINT (0 0), 'container level': 1}","{'container level': 0, 'geometry': POINT (1 1)}"
3,2020-01-01 02:00:00,STOP,TRANSFER COMPONENT TO DECK.,"{'container level': 8, 'geometry': POINT (0 0)}","{'geometry': POINT (0 0), 'container level': 2}","{'container level': 0, 'geometry': POINT (1 1)}"
4,2020-01-01 02:00:00,START,TRANSFER COMPONENT TO DECK.,"{'container level': 8, 'geometry': POINT (0 0)}","{'geometry': POINT (0 0), 'container level': 2}","{'container level': 0, 'geometry': POINT (1 1)}"
...,...,...,...,...,...,...
67,2020-01-03 18:00:00,STOP,(RE)POSITION ONTO LOCATION.,"{'container level': 0, 'geometry': POINT (0 0)}","{'geometry': POINT (1 1), 'container level': 1}","{'container level': 9, 'geometry': POINT (1 1)}"
68,2020-01-03 18:00:00,START,INSTALL COMPONENT ON SITE.,"{'container level': 0, 'geometry': POINT (0 0)}","{'geometry': POINT (1 1), 'container level': 1}","{'container level': 9, 'geometry': POINT (1 1)}"
69,2020-01-03 23:00:00,STOP,INSTALL COMPONENT ON SITE.,"{'container level': 0, 'geometry': POINT (0 0)}","{'geometry': POINT (1 1), 'container level': 0}","{'container level': 10, 'geometry': POINT (1 1)}"
70,2020-01-03 23:00:00,START,TRANSIT TO PORT.,"{'container level': 0, 'geometry': POINT (0 0)}","{'geometry': POINT (1 1), 'container level': 0}","{'container level': 10, 'geometry': POINT (1 1)}"
